# Day 18: Snailfish
https://adventofcode.com/2021/day/18

### Part 1

The modules can be found [here](advent_of_code/day_18.py).

In [1]:
from advent_of_code.utils import input_location
from advent_of_code.day_18 import SailFishNumber, str2sailfishnumber
from typing import List
from functools import reduce
from itertools import permutations

%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

In [2]:
def parse_input(filename: str) -> List[SailFishNumber]:
    with open(filename) as f:
        input: list = []
        for l in f:
            input.append(str2sailfishnumber(l.strip()))
        return input

In [3]:
numbers: List[SailFishNumber] = parse_input(input_location(day=18))
snailfish_num = reduce(lambda a, b: a + b, numbers)
snailfish_num.magnitude()  # the answer.

4243

### Part 2

In [4]:
numbers: List[SailFishNumber] = parse_input(input_location(day=18))
max((sfn1 + sfn2).magnitude() for sfn1, sfn2 in permutations(numbers, 2))

4701